In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math
import pylab as pl
#import mpld3 as ml
import math
import import_ipynb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from function_log import smape,add_tdate,get_week,density_interp,extract_date,IC_run_sim,new_compare,run_sim,stoc_eqs,Stoch_Iteration,stoch_model

import lhsmdu
import random

importing Jupyter notebook from function_log.ipynb


ModuleNotFoundError: No module named 'lhsmdu'

In [3]:
new_census = pd.read_csv('new_census.csv')
new_census['Date'] = pd.to_datetime(new_census['Date'])
new_census['ObsYear'] = new_census.Date.apply(lambda x: x.year)
new_census['ObsMonth'] = new_census.Date.apply(lambda x:x.month)

new_census.T = pd.to_numeric(new_census['T'],errors='coerce')
new_census.J = pd.to_numeric(new_census.J)
new_census.A = pd.to_numeric(new_census.A)
new_census.S = pd.to_numeric(new_census.S)

In [5]:
t_types = pd.read_csv('troop_types.csv')

rural_troops = list(t_types.loc[t_types['Type Code'] == 0]['Troop'])
rural = new_census.loc[new_census.Troop.isin(rural_troops)]

urb_troops = list(t_types.loc[t_types['Type Code'] == 1]['Troop'])
urban = new_census.loc[new_census.Troop.isin(urb_troops)]

In [6]:
from __future__ import print_function
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

In [7]:
def quit_function():
    # print to stderr, unbuffered in Python 2.
    print('Parameter set rejected', file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt

In [8]:
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [9]:
from sklearn.utils import shuffle
linparams = pd.DataFrame(data={'b_j':shuffle(np.linspace(0.254,0.344)).T,
                   'd_j':shuffle(np.linspace(0.278,0.412)).T,
                   'd_s':shuffle(np.linspace(0.136,0.296)).T,
                   'd_a':shuffle(np.linspace(0.034,0.252)).T,
                   'ae_j':shuffle(np.linspace(0.1,0.8)).T,
                   'ae_s':shuffle(np.linspace(0.1,0.8)).T,
                   'ae_a':shuffle(np.linspace(0.1,0.8)).T,
                   't_p':shuffle(np.linspace(0.15,0.7)).T,
                   'por':shuffle(np.linspace(0.1,0.6)).T,
                   'fsr':shuffle(np.linspace(0.2,0.8)).T,
                   'thresh':shuffle(np.linspace(4,6)).T,
                   'fis_thresh':shuffle(np.linspace(38,68)).T,
                   'lit_size':shuffle(np.linspace(3,5)).T,
                   'no_ev':shuffle(np.linspace(0.8,0.95)).T,
                   'rt_chg':shuffle(np.linspace(0.05,0.5)).T,
                   'fus_fis':shuffle(np.linspace(0.1,0.7)).T,
                  },columns = ['b_j', 'd_j', 'd_s', 'd_a', 'ae_j',
        'ae_s','ae_a', 't_p', 'por', 'fsr', 'thresh','fis_thresh','lit_size','no_ev', 'rt_chg','fus_fis'])

In [11]:
test = pd.read_csv('param_space.csv')
test.loc[:,'lit_size'] = round(test.lit_size)
test.loc[:,'thresh'] = round(test.thresh)

In [12]:
def sens_param(sets,iters,timeout,dataset,feat=0,range_type=0,paramset=0,start_set=0):
   
    rejects = pd.DataFrame()
   
   
    ##IMPORT parameter space
    try:
        if paramset == 0:
            if range_type==0:
                params = pd.read_csv('param_space.csv')
            else:
                params = pd.read_csv('../ABC_estimation/MatLab/{}_range_list.csv'.format(range_type))

            params.loc[:,'lit_size'] = round(params['lit_size'])
            params.loc[:,'thresh'] = round(params['thresh'])
            params.loc[:,'fis_thresh'] = round(params['fis_thresh'])
    except:
        params = paramset
   


    full_est = pd.DataFrame()
   
    param_names = ['no_ev', 'rt_chg', 'b_j', 'd_j', 'd_s', 'd_a', 'ae_j',
        'ae_s','ae_a', 't_p', 'por', 'fsr', 'thresh','fis_thresh','lit_size','fus_fis']
    feat_names = ['d_j','d_a','por','fsr','no_ev']

    ##Initialize timeout function
    @exit_after(timeout)
    def run_params(iterations,data,param_sens=0):
        
        if param_sens == 'd_j':
            par_est = new_compare(data,iters=iterations,d_j = row['d_j'])
            return par_est

        elif param_sens == 'd_a':
            par_est = new_compare(data,iters=iterations,d_a = row['d_a'])
            return par_est

        elif param_sens == 'por':
            par_est = new_compare(data,iters=iterations,por = row['por'])
            return par_est

        elif param_sens == 'fsr':
            par_est = new_compare(data,iters=iterations,fsr = row['fsr'])
            return par_est

        elif param_sens == 'no_ev':
            par_est = new_compare(data,iters=iterations,no_ev = row['no_ev'])
            return par_est

        else:
            par_est = new_compare(data,
                                   iters=iterations,
                                   years=1,
                                   no_ev = row['no_ev'],
                                   rt_chg = row['rt_chg'],
                                   fus_fis = row['fus_fis'],
                                   b_j = row['b_j'],
                                   d_j = row['d_j'],
                                   d_s = row['d_s'],
                                   d_a = row['d_a'],
                                   ae_j = row['ae_j'],
                                   ae_s = row['ae_s'],
                                   ae_a = row['ae_a'],
                                   t_p = row['t_p'],
                                   por = row['por'],
                                   fsr = row['fsr'],
                                   thresh = row['thresh'],
                                   fis_thresh = row['fis_thresh'],
                                   lit_size = row['lit_size'],
                                   dfe='off')
            return par_est

            
           
       
    ##Run parameter sets
    for index,row in params[start_set:start_set+sets].iterrows():
        if feat != 0:
            try:
                for var in feat_names:
                    if range_type == 0:
                        run_params(iterations=iters,data=dataset,param_sens=var).to_csv('../ABC_estimation/feat_runs/{}/{}_param_run_{}.csv'.format(var,var,str(index).zfill(3)))
                    else:
                        run_params(iterations=iters,data=dataset,param_sens=var).to_csv('../ABC_estimation/feat_runs/range_test/{}/{}_{}_run_{}.csv'.format(var,var,range_type,str(index).zfill(3)))
            except:
                rejects = rejects.append(row)
        else:
            try:
                run_params(iterations=iters,data=dataset).to_csv('../ABC_estimation/final_run/param_run_{}.csv'.format(str(index).zfill(3)))
            except:
                rejects = rejects.append(row)
           
    if len(rejects) >= 1:
        if feat==0:
            rejects.to_csv('../ABC_estimation/final_run/rejects.csv')
            return rejects
        else:
            rejects.to_csv('../ABC_estimation/feat_runs/feat_rejects.csv')
            return rejects
    else:
        return


In [34]:
thing = sens_param(sets=25,iters=25,timeout=300,dataset=rural,paramset=linparams, start_set=24)

Parameter set rejected
Parameter set rejected


In [4]:
linparams.to_csv('../ABC_estimation/final_run/linparams.csv')